In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_rows",130)
#pd.set_option("display.max_rows",100000)

In [2]:
df = pd.read_csv("인천시_전기차_충전소_입지분석_전처리.csv", encoding = 'CP949')

In [3]:
df.head(1)

,gid,전기차,충전소,변압기,주차장,건물1,건물2,건물3,행정기관,도서관,경제인구
0,가사462996,0,0,0,NaN,0,0,0,0,0,NaN


In [4]:
df.isnull().sum()

gid          0
전기차          0
충전소          0
변압기          0
주차장     112908
건물1          0
건물2          0
건물3          0
행정기관         0
도서관          0
경제인구    101401
dtype: int64

In [5]:
# df에서 주찾아과 경제인구에서 NaN값들 전부 0으로 채움.
df2 = df.fillna(0)
df2['주차장'] = list(map(lambda x: int(x) , df2['주차장']))
df2['경제인구'] = list(map(lambda x: int(x) , df2['경제인구']))

In [6]:
df2.iloc[:,0].unique()

array(['가사462996', '가사462997', '가사462998', ..., '다사376536', '다사376537',
       '다사376538'], dtype=object)

In [7]:
df2.columns

Index(['gid', '전기차', '충전소', '변압기', '주차장', '건물1', '건물2', '건물3', '행정기관', '도서관',
       '경제인구'],
      dtype='object')

### 변수들 상관분석

In [8]:
df2.corr()

,전기차,충전소,변압기,주차장,건물1,건물2,건물3,행정기관,도서관,경제인구
전기차,1.000000,0.097117,0.075540,0.065547,0.029916,0.107111,0.001591,0.074800,0.007935,0.145651
충전소,0.097117,1.000000,0.044398,0.016235,0.005902,0.081995,0.023997,0.012843,0.040117,0.111879
변압기,0.075540,0.044398,1.000000,0.046749,0.371472,0.122953,0.038293,0.092699,0.063687,0.171828
주차장,0.065547,0.016235,0.046749,1.000000,0.070488,0.025142,0.008444,0.036297,0.005835,0.046020
건물1,0.029916,0.005902,0.371472,0.070488,1.000000,0.104147,0.030132,0.070821,0.035293,0.202159
건물2,0.107111,0.081995,0.122953,0.025142,0.104147,1.000000,0.021284,0.037058,0.046157,0.698021
건물3,0.001591,0.023997,0.038293,0.008444,0.030132,0.021284,1.000000,0.082381,0.017269,0.018131
행정기관,0.074800,0.012843,0.092699,0.036297,0.070821,0.037058,0.082381,1.000000,0.107403,0.069961
도서관,0.007935,0.040117,0.063687,0.005835,0.035293,0.046157,0.017269,0.107403,1.000000,0.079166
경제인구,0.145651,0.111879,0.171828,0.046020,0.202159,0.698021,0.018131,0.069961,0.079166,1.000000


In [9]:
s = list(df2.columns)[1:]
s

['전기차', '충전소', '변압기', '주차장', '건물1', '건물2', '건물3', '행정기관', '도서관', '경제인구']

## MinMax 정규화

In [10]:
df3 = df2.copy()

In [11]:
for i in  s :
    df3[i] = df3[i] / (df3[i].max() - df3[i].min())

In [12]:
df3.head()

,gid,전기차,충전소,변압기,주차장,건물1,건물2,건물3,행정기관,도서관,경제인구
0,가사462996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,가사462997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,가사462998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,가사462999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,가사463995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 관공서 = 행정기관 + 도서관

In [13]:
df4 = df2.loc[:, ['gid', '전기차', '충전소', '변압기', '주차장', '건물1', '건물2', '건물3','경제인구']] 

In [14]:
df4['관공서'] = df2['행정기관'] + df2['도서관']

In [15]:
df4.head(1)

,gid,전기차,충전소,변압기,주차장,건물1,건물2,건물3,경제인구,관공서
0,가사462996,0,0,0,0,0,0,0,0,0


## 충전소 위치 데이터 

In [16]:
ch_st = pd.read_csv("3. 최종_한전_충전시설.CSV", encoding = "CP949")
ch_st.head(1)

,충전소 구분,충전소,급속,완속,지번,도로명,상세위치,등록일,시군구코드,시군구명,X좌표,Y좌표
0,업무용,강화지사,1,0,인천광역시 강화군 강화읍 갑곳리 350,"인천광역시 강화군 강화읍 강화대로 278 (갑곳리, 한국전력공사강화지점)",지사 내 주차장 북동쪽방향,2017-02-27,28710,강화군,126.499131,37.741871


In [17]:
ch_st.groupby(['시군구명']).size()

시군구명
강화군      5
계양구      6
남동구     19
동구       6
미추홀구    10
부평구     12
서구      30
연수구     30
중구      11
dtype: int64

In [18]:
# 필요 데이터들 선택
ch_sta = ch_st.iloc[:, [0,1,2,3,7,10,11]]
ch_sta.head(1)

,충전소 구분,충전소,급속,완속,등록일,X좌표,Y좌표
0,업무용,강화지사,1,0,2017-02-27,126.499131,37.741871


In [19]:
ch_sta['충전소 구분'].unique()

array(['업무용', '공용', '아파트용'], dtype=object)

## 충전소 사용 데이터 

In [20]:
# 2018년 사용이력
ch1 = pd.read_csv("4.1 충전소 사용내역_2018.CSV", encoding = "CP949")
ch1.head(1)

,용도별,충전소,주소,충전소 구축일,충전구분,충전량(kWh),시작일시
0,공용,G-Tower(경제자유구역청),인천광역시 연수구 아트센터대로 175,2017-11-17,급속,28.47,2018-04-23 09:21:05


In [21]:
# 2019년 상반기 사용이력
ch2 = pd.read_csv("4.2 충전소 사용내역_2019.CSV", encoding = "CP949")
ch2.head(1)

,용도별,충전소,주소,충전소 구축일,충전구분,충전량(kWh),시작일시
0,공용,G-Tower(경제자유구역청),인천광역시 연수구 아트센터대로 175,2017-11-17,급속,14.7,2019-01-01 16:41:50


In [22]:
# 2018년과 2019년 상반기 사용이력 결합
ch3 = pd.concat([ch1, ch2])
ch3.head(1)

,용도별,충전소,주소,충전소 구축일,충전구분,충전량(kWh),시작일시
0,공용,G-Tower(경제자유구역청),인천광역시 연수구 아트센터대로 175,2017-11-17,급속,28.47,2018-04-23 09:21:05


In [23]:
print( len(ch1), len(ch2), len(ch1) + len(ch2), len(ch3))

28598 25821 54419 54419


In [24]:
ch3.index = range(len(ch3))

In [25]:
# 필요 columns 선택
ch4 = ch3.iloc[:, [0,1,3,4,5,6]]
ch4.head(1)

,용도별,충전소,충전소 구축일,충전구분,충전량(kWh),시작일시
0,공용,G-Tower(경제자유구역청),2017-11-17,급속,28.47,2018-04-23 09:21:05


In [26]:
ch_day = ch4.iloc[:,:5]

In [27]:
ch4['시작일시'][0][:10]

'2018-04-23'

### 연도 월 일만 추출 

In [28]:
ch_day['충전일'] = list(map(lambda x : x[:10], ch4['시작일시']))

## 충전소명과 충전소 서비스 기간, 서비스 빈도 수, 서비스 일수(하루에 여러번 있어도 1로 계산됨)

In [29]:
from datetime import datetime, timedelta

datetime.strptime((pd.DataFrame(ch_day[ch_day['충전소'] == list(ch_day['충전소'].unique())[0]  ]['충전일']).max()).values[0], '%Y-%m-%d')

len(pd.Series(ch_day[ch_day['충전소'] == list(ch_day['충전소'].unique() )[0] ]['충전일']).unique())

298

In [30]:
ch_day = ch_day.sort_values(['충전소'])

In [31]:
from datetime import datetime, timedelta

ch_sta_name = []
ch_sta_dif = []
ch_ser_count = []
ch_ser_day_count = []

for  i in list(ch_day['충전소'].unique()) :
    ch_date_max = datetime.strptime( (pd.DataFrame(ch_day[ch_day['충전소'] == i]['충전일']).max()).values[0], '%Y-%m-%d')
    ch_date_min = datetime.strptime((pd.DataFrame(ch_day[ch_day['충전소'] == i ]['충전일']).min()).values[0], '%Y-%m-%d')
    ch_date_dif = (ch_date_max - ch_date_min).days + 1
    ch_ser_cot = len(pd.Series(ch_day[ch_day['충전소'] == i ]['충전일']))
    ch_ser_day_cot = (len(pd.Series(ch_day[ch_day['충전소'] == i ]['충전일']).unique()))
    
    print(i, '\n  :', ch_date_min.strftime("%Y-%m-%d"),',', ch_date_max.strftime("%Y-%m-%d"))
    print(" 차이 일수(기간) ", ch_date_dif + 1 , "일")
    print(" 서비스 빈도 수",ch_ser_cot )
    print(" 서비스 일수 ", ch_ser_day_cot," \n\n")
    
    ch_sta_name.append(i)
    ch_sta_dif.append(ch_date_dif)
    ch_ser_count.append(ch_ser_cot)
    ch_ser_day_count.append(ch_ser_day_cot)

G-Tower(경제자유구역청) 
  : 2018-04-23 , 2019-06-11
 차이 일수(기간)  416 일
 서비스 빈도 수 753
 서비스 일수  298  


LH인천본부 
  : 2018-01-01 , 2019-06-12
 차이 일수(기간)  529 일
 서비스 빈도 수 2202
 서비스 일수  502  


lh웨스턴블루힐 
  : 2018-03-21 , 2019-06-11
 차이 일수(기간)  449 일
 서비스 빈도 수 286
 서비스 일수  191  


가정뉴서울아파트 
  : 2018-03-22 , 2019-05-17
 차이 일수(기간)  423 일
 서비스 빈도 수 29
 서비스 일수  29  


간석래미안자이 
  : 2018-02-05 , 2019-06-12
 차이 일수(기간)  494 일
 서비스 빈도 수 111
 서비스 일수  81  


갈산이안 아파트 
  : 2018-08-21 , 2018-08-21
 차이 일수(기간)  2 일
 서비스 빈도 수 1
 서비스 일수  1  


강화군청 
  : 2018-01-02 , 2019-06-11
 차이 일수(기간)  527 일
 서비스 빈도 수 953
 서비스 일수  408  


강화지사 
  : 2018-01-01 , 2019-06-06
 차이 일수(기간)  523 일
 서비스 빈도 수 444
 서비스 일수  294  


검단아이파크아파트 
  : 2018-01-20 , 2019-06-09
 차이 일수(기간)  507 일
 서비스 빈도 수 79
 서비스 일수  72  


검단자이1단지 아파트 
  : 2018-01-01 , 2019-06-12
 차이 일수(기간)  529 일
 서비스 빈도 수 704
 서비스 일수  395  


검암2차풍림아파트 
  : 2018-03-26 , 2019-06-04
 차이 일수(기간)  437 일
 서비스 빈도 수 70
 서비스 일수  53  


계양구청 
  : 2018-05-07 , 2019-06-12
 차이 일수(기간)  403 일
 

청라롯데캐슬 아파트 
  : 2018-01-12 , 2019-06-12
 차이 일수(기간)  518 일
 서비스 빈도 수 906
 서비스 일수  387  


청라린스트라우스 아파트 
  : 2018-04-14 , 2019-06-11
 차이 일수(기간)  425 일
 서비스 빈도 수 504
 서비스 일수  198  


청라반도유보라2.0 아파트 
  : 2018-03-06 , 2019-06-11
 차이 일수(기간)  464 일
 서비스 빈도 수 899
 서비스 일수  345  


청라상록힐스테이트10블럭 
  : 2018-01-02 , 2019-06-11
 차이 일수(기간)  527 일
 서비스 빈도 수 127
 서비스 일수  103  


청라엑슬루타워 아파트 
  : 2018-03-28 , 2019-06-07
 차이 일수(기간)  438 일
 서비스 빈도 수 81
 서비스 일수  73  


청라우미린아파트 
  : 2018-06-25 , 2018-09-27
 차이 일수(기간)  96 일
 서비스 빈도 수 5
 서비스 일수  3  


청라웰카운티17B 아파트 
  : 2018-01-09 , 2019-06-11
 차이 일수(기간)  520 일
 서비스 빈도 수 78
 서비스 일수  62  


청라웰카운티19단지 
  : 2018-06-26 , 2019-06-12
 차이 일수(기간)  353 일
 서비스 빈도 수 267
 서비스 일수  186  


청라자이 아파트 
  : 2018-01-01 , 2019-06-11
 차이 일수(기간)  528 일
 서비스 빈도 수 424
 서비스 일수  301  


청라중흥S클래스2단지 아파트 
  : 2018-01-02 , 2019-06-09
 차이 일수(기간)  525 일
 서비스 빈도 수 135
 서비스 일수  120  


청라중흥s-클래스1단지 아파트 
  : 2018-01-02 , 2019-06-10
 차이 일수(기간)  526 일
 서비스 빈도 수 569
 서비스 일수  358  


청라푸르지오 아파트

In [32]:
ch_sta_info = pd.DataFrame( [ch_sta_name, ch_sta_dif, ch_ser_count, ch_ser_day_count], 
                           index = ['충전소', '서비스 기간', '서비스 빈도', '서비스 일수']).T
ch_sta_info

,충전소,서비스 기간,서비스 빈도,서비스 일수
0,G-Tower(경제자유구역청),415,753,298
1,LH인천본부,528,2202,502
2,lh웨스턴블루힐,448,286,191
3,가정뉴서울아파트,422,29,29
4,간석래미안자이,493,111,81
5,갈산이안 아파트,1,1,1
6,강화군청,526,953,408
7,강화지사,522,444,294
8,검단아이파크아파트,506,79,72
9,검단자이1단지 아파트,528,704,395


In [33]:
ch_sta_info2 = ch_sta_info.sort_values(['충전소'])
ch_sta_info2.index = range(len(ch_sta_info2))
ch_sta_info2

,충전소,서비스 기간,서비스 빈도,서비스 일수
0,G-Tower(경제자유구역청),415,753,298
1,LH인천본부,528,2202,502
2,lh웨스턴블루힐,448,286,191
3,가정뉴서울아파트,422,29,29
4,간석래미안자이,493,111,81
5,갈산이안 아파트,1,1,1
6,강화군청,526,953,408
7,강화지사,522,444,294
8,검단아이파크아파트,506,79,72
9,검단자이1단지 아파트,528,704,395


## 충전소별 총 충전량 

In [34]:
ch_sta_tot = ch_day.groupby(['충전소']).sum()
ch_sta_tot.columns = ['총 충전량']
ch_sta_tot

,총 충전량
충전소,
G-Tower(경제자유구역청),16950.370
LH인천본부,44845.700
lh웨스턴블루힐,9306.430
가정뉴서울아파트,487.060
간석래미안자이,2123.190
갈산이안 아파트,0.540
강화군청,19855.630
강화지사,9523.120
검단아이파크아파트,1286.650


In [35]:
ch_sta_info3 = pd.merge( ch_sta_info2, ch_sta_tot, how = "left", on = '충전소')
ch_sta_info3

,충전소,서비스 기간,서비스 빈도,서비스 일수,총 충전량
0,G-Tower(경제자유구역청),415,753,298,16950.370
1,LH인천본부,528,2202,502,44845.700
2,lh웨스턴블루힐,448,286,191,9306.430
3,가정뉴서울아파트,422,29,29,487.060
4,간석래미안자이,493,111,81,2123.190
5,갈산이안 아파트,1,1,1,0.540
6,강화군청,526,953,408,19855.630
7,강화지사,522,444,294,9523.120
8,검단아이파크아파트,506,79,72,1286.650
9,검단자이1단지 아파트,528,704,395,12165.690


## 충전소별 모든 정보

In [36]:
ch_tot_info = pd.merge(ch_sta, ch_sta_info3, how = "left", on = '충전소')
ch_tot_info

,충전소 구분,충전소,급속,완속,등록일,X좌표,Y좌표,서비스 기간,서비스 빈도,서비스 일수,총 충전량
0,업무용,강화지사,1,0,2017-02-27,126.499131,37.741871,522,444,294,9523.120
1,공용,강화군청,1,0,2017-09-19,126.487733,37.746131,526,953,408,19855.630
2,공용,신문리 공영주차장,1,0,2018-09-11,126.479158,37.746535,131,101,68,2310.310
3,공용,온수리 제1공영주차장,1,0,2018-09-11,126.489502,37.641595,113,36,28,591.570
4,공용,마니산국민관광지,1,0,2017-09-19,126.423797,37.632456,523,471,304,9419.810
5,공용,홈플러스 계산점,1,0,2017-11-17,126.736184,37.539470,356,397,226,7999.910
6,공용,계양구청,4,0,2017-11-17,126.737757,37.537442,402,1163,375,23714.350
7,아파트용,초정마을동아 아파트,1,1,2017-08-12,126.740800,37.538150,301,69,66,1018.360
8,공용,하나로마트 인천점,1,0,2017-11-17,126.728470,37.532639,392,263,199,6571.410
9,아파트용,작전동보2차 아파트,1,1,2017-08-12,126.731319,37.526047,421,76,60,603.720


### NaN값들 0으로 대체 (시설등록이 되어있지만 사용 이력이 하나도 없는 경우)

In [37]:
ch_tot_info[ch_tot_info['총 충전량'].isnull() == 1]

,충전소 구분,충전소,급속,완속,등록일,X좌표,Y좌표,서비스 기간,서비스 빈도,서비스 일수,총 충전량
55,업무용,주안변전소,0,1,2018-07-06,126.698857,37.472146,NaN,NaN,NaN,NaN
121,업무용,제물포지사,0,4,2016-10-27,126.633297,37.454909,NaN,NaN,NaN,NaN


In [38]:
ch_tot_info[ch_tot_info['총 충전량'].isnull() == 1] = ch_tot_info[ch_tot_info['총 충전량'].isnull() == 1].fillna(0)

In [39]:
ch_tot_info

,충전소 구분,충전소,급속,완속,등록일,X좌표,Y좌표,서비스 기간,서비스 빈도,서비스 일수,총 충전량
0,업무용,강화지사,1,0,2017-02-27,126.499131,37.741871,522,444,294,9523.120
1,공용,강화군청,1,0,2017-09-19,126.487733,37.746131,526,953,408,19855.630
2,공용,신문리 공영주차장,1,0,2018-09-11,126.479158,37.746535,131,101,68,2310.310
3,공용,온수리 제1공영주차장,1,0,2018-09-11,126.489502,37.641595,113,36,28,591.570
4,공용,마니산국민관광지,1,0,2017-09-19,126.423797,37.632456,523,471,304,9419.810
5,공용,홈플러스 계산점,1,0,2017-11-17,126.736184,37.539470,356,397,226,7999.910
6,공용,계양구청,4,0,2017-11-17,126.737757,37.537442,402,1163,375,23714.350
7,아파트용,초정마을동아 아파트,1,1,2017-08-12,126.740800,37.538150,301,69,66,1018.360
8,공용,하나로마트 인천점,1,0,2017-11-17,126.728470,37.532639,392,263,199,6571.410
9,아파트용,작전동보2차 아파트,1,1,2017-08-12,126.731319,37.526047,421,76,60,603.720


## 필요에 따른 전처리 

In [40]:
# 복사
ch_tot_info_temp = ch_tot_info.copy()
ch_tot_info_temp

,충전소 구분,충전소,급속,완속,등록일,X좌표,Y좌표,서비스 기간,서비스 빈도,서비스 일수,총 충전량
0,업무용,강화지사,1,0,2017-02-27,126.499131,37.741871,522,444,294,9523.120
1,공용,강화군청,1,0,2017-09-19,126.487733,37.746131,526,953,408,19855.630
2,공용,신문리 공영주차장,1,0,2018-09-11,126.479158,37.746535,131,101,68,2310.310
3,공용,온수리 제1공영주차장,1,0,2018-09-11,126.489502,37.641595,113,36,28,591.570
4,공용,마니산국민관광지,1,0,2017-09-19,126.423797,37.632456,523,471,304,9419.810
5,공용,홈플러스 계산점,1,0,2017-11-17,126.736184,37.539470,356,397,226,7999.910
6,공용,계양구청,4,0,2017-11-17,126.737757,37.537442,402,1163,375,23714.350
7,아파트용,초정마을동아 아파트,1,1,2017-08-12,126.740800,37.538150,301,69,66,1018.360
8,공용,하나로마트 인천점,1,0,2017-11-17,126.728470,37.532639,392,263,199,6571.410
9,아파트용,작전동보2차 아파트,1,1,2017-08-12,126.731319,37.526047,421,76,60,603.720


### 아파트, 비아파트 구분 필요 시  

In [41]:
'''cti_aprt_info = ch_tot_info[ ch_tot_info['충전소 구분'] == '아파트용']
cti_noaprt_info = ch_tot_info[ ch_tot_info['충전소 구분'] != '아파트용']
print( len(cti_aprt_info), len(cti_noaprt_info), len(cti_aprt_info) + len(cti_noaprt_info) , len(ch_tot_info))'''

"cti_aprt_info = ch_tot_info[ ch_tot_info['충전소 구분'] == '아파트용']\ncti_noaprt_info = ch_tot_info[ ch_tot_info['충전소 구분'] != '아파트용']\nprint( len(cti_aprt_info), len(cti_noaprt_info), len(cti_aprt_info) + len(cti_noaprt_info) , len(ch_tot_info))"

### 필요 Columns만 추출 

In [42]:
ch_tot_info_temp = ch_tot_info.iloc[:, [1,0,5,6,7,8,9,10] ]
ch_tot_info_temp   

,충전소,충전소 구분,X좌표,Y좌표,서비스 기간,서비스 빈도,서비스 일수,총 충전량
0,강화지사,업무용,126.499131,37.741871,522,444,294,9523.120
1,강화군청,공용,126.487733,37.746131,526,953,408,19855.630
2,신문리 공영주차장,공용,126.479158,37.746535,131,101,68,2310.310
3,온수리 제1공영주차장,공용,126.489502,37.641595,113,36,28,591.570
4,마니산국민관광지,공용,126.423797,37.632456,523,471,304,9419.810
5,홈플러스 계산점,공용,126.736184,37.539470,356,397,226,7999.910
6,계양구청,공용,126.737757,37.537442,402,1163,375,23714.350
7,초정마을동아 아파트,아파트용,126.740800,37.538150,301,69,66,1018.360
8,하나로마트 인천점,공용,126.728470,37.532639,392,263,199,6571.410
9,작전동보2차 아파트,아파트용,126.731319,37.526047,421,76,60,603.720


### 총 충전량을 필요 기준에 따라 계산

In [43]:
ch_tot_info_temp['총/기간'] = [0]*len(ch_tot_info_temp)
ch_tot_info_temp['총/빈도'] = [0]*len(ch_tot_info_temp)
ch_tot_info_temp['총/일수'] = [0]*len(ch_tot_info_temp)

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [44]:
ch_tot_info_temp

,충전소,충전소 구분,X좌표,Y좌표,서비스 기간,서비스 빈도,서비스 일수,총 충전량,총/기간,총/빈도,총/일수
0,강화지사,업무용,126.499131,37.741871,522,444,294,9523.120,0,0,0
1,강화군청,공용,126.487733,37.746131,526,953,408,19855.630,0,0,0
2,신문리 공영주차장,공용,126.479158,37.746535,131,101,68,2310.310,0,0,0
3,온수리 제1공영주차장,공용,126.489502,37.641595,113,36,28,591.570,0,0,0
4,마니산국민관광지,공용,126.423797,37.632456,523,471,304,9419.810,0,0,0
5,홈플러스 계산점,공용,126.736184,37.539470,356,397,226,7999.910,0,0,0
6,계양구청,공용,126.737757,37.537442,402,1163,375,23714.350,0,0,0
7,초정마을동아 아파트,아파트용,126.740800,37.538150,301,69,66,1018.360,0,0,0
8,하나로마트 인천점,공용,126.728470,37.532639,392,263,199,6571.410,0,0,0
9,작전동보2차 아파트,아파트용,126.731319,37.526047,421,76,60,603.720,0,0,0


In [45]:
# 서비스 기간이 값이 0인 행의 index
a = set(ch_tot_info_temp[ ch_tot_info_temp['서비스 기간'] == 0 ].index)

# 서비스 빈도가 값이 0인 행의 index
b = set(ch_tot_info_temp[ ch_tot_info_temp['서비스 기간'] == 0 ].index)

# 서비스 일 수가 값이 0인 행의 index
c = set(ch_tot_info_temp[ ch_tot_info_temp['서비스 기간'] == 0 ].index) 

# 위에서 구한 0인 값들 제외하고 기준에 따른 값들 구하기

ch_tot_info_temp.loc[ list(set(ch_tot_info_temp['서비스 기간'].index) - a), '총/기간'] = (
    ch_tot_info_temp.loc[ list(set(ch_tot_info_temp['서비스 기간'].index) - a), '총 충전량'] / ch_tot_info_temp.loc[ list(set(ch_tot_info_temp['서비스 기간'].index) - a), '서비스 기간']
)
    
ch_tot_info_temp.loc[ list(set(ch_tot_info_temp['서비스 빈도'].index) - b), '총/빈도'] = (
    ch_tot_info_temp.loc[ list(set(ch_tot_info_temp['서비스 빈도'].index) - b), '총 충전량'] / ch_tot_info_temp.loc[ list(set(ch_tot_info_temp['서비스 빈도'].index) - b), '서비스 빈도'] 
)
    
ch_tot_info_temp.loc[ list(set(ch_tot_info_temp['서비스 일수'].index) - c), '총/일수'] = (
    ch_tot_info_temp.loc[ list(set(ch_tot_info_temp['서비스 일수'].index) - c), '총 충전량'] / ch_tot_info_temp.loc[ list(set(ch_tot_info_temp['서비스 일수'].index) - c), '서비스 일수'] 
)

ch_tot_info_temp

,충전소,충전소 구분,X좌표,Y좌표,서비스 기간,서비스 빈도,서비스 일수,총 충전량,총/기간,총/빈도,총/일수
0,강화지사,업무용,126.499131,37.741871,522,444,294,9523.120,18.243525,21.448468,32.391565
1,강화군청,공용,126.487733,37.746131,526,953,408,19855.630,37.748346,20.834869,48.665760
2,신문리 공영주차장,공용,126.479158,37.746535,131,101,68,2310.310,17.635954,22.874356,33.975147
3,온수리 제1공영주차장,공용,126.489502,37.641595,113,36,28,591.570,5.235133,16.432500,21.127500
4,마니산국민관광지,공용,126.423797,37.632456,523,471,304,9419.810,18.011109,19.999597,30.986217
5,홈플러스 계산점,공용,126.736184,37.539470,356,397,226,7999.910,22.471657,20.150907,35.397832
6,계양구청,공용,126.737757,37.537442,402,1163,375,23714.350,58.990920,20.390671,63.238267
7,초정마을동아 아파트,아파트용,126.740800,37.538150,301,69,66,1018.360,3.383256,14.758841,15.429697
8,하나로마트 인천점,공용,126.728470,37.532639,392,263,199,6571.410,16.763801,24.986350,33.022161
9,작전동보2차 아파트,아파트용,126.731319,37.526047,421,76,60,603.720,1.434014,7.943684,10.062000


In [48]:
#ch_tot_info_temp.to_csv("모든 충전소별 정보.csv", encoding = "CP949", index = False)

# 충전소가 있는 곳들에 데이터만 추출하여 충전량, 기간, 빈도, 일수에 대하여
# 각각 상관분석 실시!